# P&D - Cientista de Dados - ChatBot

## Motivação

Grande parte dos dados são desestruturados: desde a descrição de produtos até conversas referentes ao atendimento são salvos como texto nos nossos bancos de dados. Temos enorme interesse em utilizar nosso volume de dados para criação de processos automáticos dentro da empresa.

## Desafio

O intuito desse desafio é entender como você abordaria essa problemática de processamento e compreensão de texto para alguns casos reduzidos de conversas entre nossos clientes e a empresa:
- Conversa cotidiana
- Suporte pós venda
- Conversa técnica

Uma pequena base de dados foi criada para efeito de ilustração e pode ser usada com referência.

Na avaliação desse desafio serão considerados:
- As hipóteses levantadas
- O rigor, a consistência e a escalabilidade da solução
- O domínio dos conhecimentos utilizados
- A escolha de bibliotécas e/ou frameworks utilizados

Não estamos procurando a resposta correta, mas sim buscando entender a sua metodologia e o seu entendimento do problema. Quaisquer dúvidas, fique a vontade para entrar em contato com: **luigi.tedesco@luizalabs.com**

O esqueleto abaixo foi montado para guiar a resolução do problema, mas fique a vontade para ignorá-lo caso tenha vontade.

In [220]:
conversations = [

    # small talk
    [
        {'user': 'user_1', 'message': 'Oi, como você tá?'                            , 'status': None},
        {'user': 'user_2', 'message': 'tudo certinho e vc?   '                       , 'status': None},
        {'user': 'user_1', 'message': ' Eu to bemm!!!'                               , 'status': None},
        {'user': 'user_1', 'message': 'tem alguma promoção   hoje?'                  , 'status': None},
        {'user': 'user_2', 'message': 'hj? temos umatv 50" \n '                      , 'status': None},
    ],
    
    # customer service
    [
        {'user': 'user_1', 'message': 'Cadê o iphone?!'                              , 'status': 'payment_approved'},
        {'user': 'attd_1', 'message': 'Olá, o seu pagamento já foi aprovado'         , 'status': 'payment_approved'},
        {'user': 'attd_1', 'message': 'Quer dizer que o seu produto está a caminho'  , 'status': 'payment_approved'},
        {'user': 'user_1', 'message': 'Mas já faz cinco dias'                        , 'status': 'payment_approved'},
        {'user': 'attd_1', 'message': 'A nossa política de entrega são 5 dias úteis' , 'status': 'payment_approved'},
        {'user': 'user_1', 'message': 'Ahh é verdade xD'                             , 'status': 'payment_approved'},
    ],
    
    # sale
    [
        {'user': 'user_1', 'message': 'cade o iphone 10?'                            , 'status': 'shopping'},
        {'user': 'sell_1', 'message': 'Oi, o Iphone X está fora de estoque'          , 'status': 'shopping'},
        {'user': 'user_1', 'message': 'Huum, o que vcs tem disponível'               , 'status': 'shoppinng'},
        {'user': 'sell_1', 'message': 'Olha, temos o iphone X plus e o samsung s8'   , 'status': 'shopping'},
        {'user': 'user_1', 'message': 'O samsung é melhor que o iphone?'             , 'status': 'shopping'},
        {'user': 'sell_1', 'message': 'Eles são diferentes, mas são os melhores'     , 'status': 'shopping'},
    ],
]

o Natural Language Tollkit foi utilizado para:
- Criação do classificador de sentimentos
- Criação da resposta atribuída ao bot

O módulo csv foi utilizado para:
- Ler sentenças previamente classificadas como: Positivo, Neutro, Negativo para utilização no classificador de sentimentos.

In [221]:
import nltk
import csv
from nltk.classify import NaiveBayesClassifier

O trecho de código abaixo criou a base de treinamento para as respostas atribuídas ao bot. Algumas Classes foram criadas a partir do esqueleto sugerido no desafio e solicitações de conversas que partiram dos Clientes.
Uma base de dados com um número expressivo de sentenças melhoraria o desempenho do classificador.

In [222]:
def trainingDatabase():
    client = [
                {'user': 'user_1', 'message': 'Oi, como você tá?', 'status': 'saudacao'},
                {'user': 'user_1', 'message': 'Eu to bemm!!!', 'status': 'saudacao'},
                {'user': 'user_1', 'message': 'tem alguma promoção   hoje?', 'status': 'promocao'},
                {'user': 'user_1', 'message': 'Cadê o iphone?!', 'status': 'prazo'},
                {'user': 'user_1', 'message': 'Mas já faz cinco dias', 'status': 'reclamacao'},
                {'user': 'user_1', 'message': 'cade o iphone 10?', 'status': 'disponibilidade'},
                {'user': 'user_1', 'message': 'Huum, o que vcs tem disponível', 'status': 'disponibilidade'},
                {'user': 'user_1', 'message': 'O samsung é melhor que o iphone?', 'status': 'duvidatecnica'},
                {'user': 'user_1', 'message': 'Ahh é verdade xD', 'status': 'satisfacao'}
    ]
    sau = readClient(client, 'saudacao')
    pro = readClient(client, 'promocao') 
    pra = readClient(client, 'prazo')
    rec = readClient(client, 'reclamacao')
    dis = readClient(client, 'disponibilidade')
    dte = readClient(client, 'duvidatecnica')    
    sat = readClient(client, 'satisfacao')
    return sau + pro + pra + rec + dis + dte + sat

A função abaixo criou a base de treinamento para o classificador de sentimentos. Foi utilizado um arquivo ('Tweets.csv') com ~8000 tweets classificados como Positivo, Negativo e Neutro. A opção por tweets está relacionada ao tamanho e objetividade das respostas, que se aproximam da proposta do desafio. Uma base com sentenças no domínio de atendimento, com os mesmos grupos (Positivo, Negativo e Neutro), poderia aumentar a eficiência e eficácia do classificador.

In [223]:
def trainingDatabaseSentiment(file = 'Tweets.csv'):
    pos = readTweets(file, 'Positivo')
    neg = readTweets(file, 'Negativo') 
    neu = readTweets(file, 'Neutro')    
    return pos + neg + neu

As funções readClient e readTweets foram responsáveis pela criação dos vetores de características das bases de treinamento do classificador para o Bot e Sentimento, respectivamente.

In [224]:
def readClient(array, status):
    bin = []
    for i in array:        
        if(i['status'].strip() == status):
            bin.append([format_sentence(i['message'].strip()), status])
    return bin

In [225]:
def readTweets(file, sentiment):
    arrayTweets = []
    with open(file, 'rt', encoding="utf8") as csvfile:
        spamreader = csv.reader(csvfile, delimiter=',', quotechar='"')
        for row in spamreader:
            if(row[9].strip() == sentiment):
                arrayTweets.append([format_sentence(row[2].strip()), sentiment])                
    return arrayTweets

O trecho de código abaixo criou os tokens por palavras únicas, que posteriormente foram utilizados nos vetores de características.
Outras técnicas poderiam ser utilizadas como: Part of Speach, Entity recognition, Lematização, Stemmer e WordNet. A precisão e revocação dos classificadores poderiam alcançar números mais favoráveis.

In [226]:
def format_sentence(sent):
    return({word: True for word in nltk.word_tokenize(sent)})

Funções abaixo criaram os classificadores para o Bot e Sentimento, respectivamente, a partir das bases de treinamento.
O classificador Naive Bayes foi utilizado. Ele apresenta bom desempenho ao ser utilizado para procesamento de linguagem natural, no entanto, experimentos com classificadores lineares e não lineares devem ser realizados.

In [227]:
def createClassifier():
    training = trainingDatabase()
    classifier = NaiveBayesClassifier.train(training)
    return classifier

In [228]:
def createClassifierSentiment():
    training = trainingDatabaseSentiment('Tweets.csv')
    classifier = NaiveBayesClassifier.train(training)    
    return classifier

In [229]:
def predict(message, **kwargs):
    return 'bip bip'

A função "predictChat" não foi utilizada, no entanto mostra como as respostas do Bot poderiam ser processadas a partir de expressões regulares.

In [230]:
def predictChat(message, **kwargs):
    from nltk.chat.util import Chat, reflections
    
    pairs = (
      (r'Oi(.*)?(.*)',
      ( "Tudo certinho e vc? Em que posso ajudá-lo?",
        "Tudo ótimo e você? Em que posso ajudá-lo?")),

      (r'(.*)promoção(.*)', 
      ("hj? temos umatv 50",
       "Sim, Temos uma TV 50 polegadas")),

      (r'Cadê(.*)',
      ( "Olá, o seu pagamento já foi aprovado. Quer dizer que o seu produto está a caminho!",
        "Ótimo, o seu pagamento já foi aprovado. Quer dizer que o seu produto está a caminho")),

      (r'(.*)dias(.*)',
      ( "A nossa política de entrega são 5 dias úteis",
        "Chegará em breve! A nossa política de entrega são 5 dias úteis")), 

      (r'(.*)iphone 10(.*)',
      ( "Oi, o Iphone X está fora de estoque",
        "Oi, o Iphone X está fora de estoque. Temos outros produtos disponíveis!")),

      (r'(.*)disponível(.*)',
      ( "Olha, temos o iphone X plus e o samsung s8",
        "Olha, temos o iphone X plus e o samsung s8. São ótimos")),

      (r'(.*)samsung(.*)iphone(.*)',
      ( "São diferentes, mas são ótimos!",
        "Ambos estão em promoção e são ótimos!")),
        
      (r'(.*)Obrigado(.*)',
      ( "Muito bom conversar com você!",
        "Volte sempre!")),
        
      (r'(.*)',
      ( "Em que posso ajudá-lo?",
        "Deseja algum produto?"))        
    )
    
    reflections = {
        "eu sou"    : "você é",
        "eu era"    : "você era",
        "eu"        : "você",        
        "meu"       : "seu",
        "você é"    : "eu sou",
        "você era"  : "eu era",
        "seu"       : "meu",
        "seus"      : "meus",
        "você"      : "eu"
    }

    chatbot = Chat(pairs, reflections)    
    
    return chatbot.respond(message)
    

A função "predictChatNaive" foi utilizada em conjunto com o classificador para respostas atribuídas ao Bot. Os pares criados estão em concordância com as Classes utilizadas no classificador.

In [231]:
def predictChatNaive(message, **kwargs):
    
    from nltk.chat.util import Chat, reflections
    
    pairs = (
      (r'saudacao',
      ( "tudo certinho e vc?",
        "Tudo ótimo! E você")),

      (r'promocao', 
      ("hj? temos umatv 50",
       "hj? temos iphone X")),      

      (r'prazo',
      ( "Olá, o seu pagamento já foi aprovado",
        "Pagamento já foi aprovado. Quer dizer que o seu produto está a caminho")),
      
      (r'reclamacao', 
      ("A nossa política de entrega são 5 dias úteis",
       "Está dentro do prazo de entrega")),
        
      (r'disponibilidade', 
      ("Oi, o Iphone X está fora de estoque",
       "Olha, temos o iphone X plus e o samsung s8")),
        
      (r'duvidatecnica', 
      ("Eles são diferentes, mas são os melhores",
       "São os melhores aparelhos que nós temos!")),
        
      (r'satisfacao', 
      ("Muito bom conversar com você! Até a próxima.",
       "Esperamos vê-lo novamente! Tchau.")),  
      
      (r'nervoso', 
      ("Fácil de resolver!",
       "Tranquilo!"))
        
    )
    
    reflections = {
        "eu sou"    : "você é",
        "eu era"    : "você era",
        "eu"        : "você",        
        "meu"       : "seu",
        "você é"    : "eu sou",
        "você era"  : "eu era",
        "seu"       : "my",
        "seus"      : "meus",
        "você"      : "eu"
    }

    chatbot = Chat(pairs, reflections)    
    
    return chatbot.respond(message)

In [232]:
def evaluate(predicted, human):
    return 1

In [233]:
# evaluation
score = []
for conversation in conversations:
    cnt = 0
    bot = []
    human = []

    for message in conversation:
        if message['user'] == 'user_1':
            bot.append({
                'user': 'bot',
                'message': predictChat(**message),
                'status': '',
            })
        else:
            human.append(message)
    
    #print(bot)
    score.append(evaluate(bot, human))
    
print(score, sum(score)/len(score))                              

[1, 1, 1] 1.0


O trecho de código abaixo foi modificado e incluiu o acionamento do classificador de sentimento, que sugere respostas a partir de questionamentos do usuário com sentimento negativo.
Também contempla as respostas do Bot utilizando o classificador e o mecanismo de Chat.

In [234]:
# have fun
text_size = 100
bubbles = ('>>>', '<<<')

cf = createClassifier()
cfsentiment = createClassifierSentiment()

user = 'start'
while True:
    # user input
    print(bubbles[0], end=' ')
    user = input()
    print('\n')    

    # exit command
    if user == 'exit':
        break   

    # bot response
    if(cfsentiment.classify(format_sentence(user)) == 'Negativo'):
        bot = predictChatNaive('nervoso') + ' ' + predictChatNaive(cf.classify(format_sentence(user)))       
    else:
        bot = predictChatNaive(cf.classify(format_sentence(user)))
        
    text_box = (
        text_size 
        - len(bubbles[0])
        - len(bubbles[1])
        - 2
    ) + 1

    split = int(len(bot)/text_box)
    if split > 0:
        start = 0
        end = text_box
        for line in range():
            start += text_box
            end += text_box
            bot_msg = bot[start:end]
            print(
                ' ' * len(bubbles[0]),
                bot_msg,
                ' ' * (text_box - len(bot_msg)),
                bubbles[1]
            )
    else:
        print(
            ' ' * (text_size - len(bubbles[0])),
            bot, bubbles[1]
        )
    print('\n')

>>> Olá tudo bem?


                                                                                                  tudo certinho e vc? <<<


>>> tem alguma promoção   hoje?


                                                                                                  hj? temos iphone X <<<


>>> Cadê o iphone?!


                                                                                                  Tranquilo! Olá, o seu pagamento já foi aprovado <<<


>>> Mas já faz cinco dias


                                                                                                  Está dentro do prazo de entrega <<<


>>> Ahh é verdade xD


                                                                                                  Esperamos vê-lo novamente! Tchau. <<<


>>> cade o iphone 10?


                                                                                                  Olha, temos o iphone X plus e o samsung s8 <<<


>>> Huum, o que vcs tem dispo